
# 📰 Task 1 — RPP News Retrieval with LangChain + Chroma

Este notebook construye un buscador semántico de noticias de **RPP Perú** siguiendo la rúbrica:

**Load → Tokenize → Embed → Store → Query**


## 0) Instalación de dependencias

In [1]:

!pip -q install feedparser tiktoken sentence-transformers langchain langchain-community langchain-chroma pandas matplotlib


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 1) Imports y configuración

In [2]:

import os
from typing import List, Dict, Any

import feedparser
import pandas as pd
import tiktoken

from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
from langchain_chroma import Chroma

os.makedirs("outputs", exist_ok=True)
print("Entorno listo ✅")


Entorno listo ✅


## 2) Parámetros generales

In [3]:

RSS_URL = "https://rpp.pe/rss"
N_ITEMS = 50
MAX_TOKENS = 800
ENCODING = "cl100k_base"


## 3) Funciones de tokenización y chunking

In [4]:

enc = tiktoken.get_encoding(ENCODING)

def count_tokens(text: str) -> int:
    return len(enc.encode(text or ""))

def chunk_text(text: str, max_tokens: int = MAX_TOKENS) -> List[str]:
    tokens = enc.encode(text or "")
    if len(tokens) <= max_tokens:
        return [text]
    chunks, step = [], max_tokens
    for i in range(0, len(tokens), step):
        chunk_tokens = tokens[i:i+step]
        chunks.append(enc.decode(chunk_tokens))
    return chunks


## 4) Cargar noticias desde el RSS de RPP

In [5]:

def load_rss_rpp(rss_url: str = RSS_URL, limit: int = N_ITEMS) -> pd.DataFrame:
    feed = feedparser.parse(rss_url)
    items = []
    for entry in feed.entries[:limit]:
        items.append({
            "title": entry.get("title", ""),
            "description": entry.get("summary", ""),
            "link": entry.get("link", ""),
            "date_published": entry.get("published", "")
        })
    return pd.DataFrame(items, columns=["title","description","link","date_published"])

df_raw = load_rss_rpp()
print(f"Noticias cargadas: {len(df_raw)}")
df_raw.head(10)


Noticias cargadas: 50


,title,description,link,date_published
0,Poder Judicial evalúa pedido fiscal de detenci...,Los policías Luis Magallanes y Omar Saavedra a...,https://rpp.pe/politica/judiciales/caso-eduard...,"Fri, 17 Oct 2025 21:30:15 -0500"
1,Cúal fue el último temblor en México hoy 17 de...,Cuál es el ultimo temblor en México y CDMX reg...,https://rpp.pe/mundo/mexico/cual-fue-el-ultimo...,"Thu, 16 Oct 2025 06:48:35 -0500"
2,"Temblor en Perú, hoy 17 de octubre: magnitud y...",Actualización EN VIVO del último sismo en Perú...,https://rpp.pe/lima/desastres-naturales/temblo...,"Thu, 16 Oct 2025 02:24:58 -0500"
3,Historial de reportes de sismos magnitud y epi...,¿Cuál fue el último Temblor en Chile hoy 17 de...,https://rpp.pe/mundo/chile/temblor-en-chile-ho...,"Wed, 15 Oct 2025 17:09:45 -0500"
4,Temblor en Chile hoy 17 de octubre: Epicentro ...,¿Cuál fue el último Temblor en Chile hoy 17 de...,https://rpp.pe/mundo/chile/temblor-en-chile-ho...,"Fri, 17 Oct 2025 20:09:45 -0500"
5,"Imagine Dragons en Lima: setlist, horarios y t...",La banda de pop rock llegará por primera vez a...,https://rpp.pe/musica/conciertos/imagine-drago...,"Fri, 17 Oct 2025 19:37:24 -0500"
6,Señor de los Milagros: fe y tradición que se m...,"Cada octubre, innumerables fieles acompañan al...",https://rpp.pe/peru/actualidad/senor-de-los-mi...,"Fri, 17 Oct 2025 19:28:45 -0500"
7,Cusco: Procuraduría Anticorrupción advierte qu...,La institución explicó que cada abogado asume ...,https://rpp.pe/peru/cusco/cusco-procuraduria-a...,"Fri, 17 Oct 2025 19:23:30 -0500"
8,“Kinra” y “Cuadrilátero”: Dos películas peruan...,Dos producciones peruanas fueron elegidas para...,https://rpp.pe/campanas/valor-compartido/kinra...,"Fri, 17 Oct 2025 18:52:23 -0500"
9,Señor de los Milagros: conoce los desvíos del ...,La Autoridad de Transporte Urbano de Lima y Ca...,https://rpp.pe/lima/actualidad/senor-de-los-mi...,"Fri, 17 Oct 2025 19:20:14 -0500"


## 5) Construcción de documentos y embeddings

In [6]:

embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

class STEmbeddings(Embeddings):
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return embed_model.encode(texts, normalize_embeddings=True).tolist()
    def embed_query(self, text: str) -> List[float]:
        return embed_model.encode([text], normalize_embeddings=True).tolist()[0]

def build_docs(df: pd.DataFrame) -> List[Dict[str, Any]]:
    docs = []
    for i, row in df.iterrows():
        combined_text = (row["title"] or "") + " | " + (row["description"] or "")
        pieces = chunk_text(combined_text, max_tokens=MAX_TOKENS)
        for j, piece in enumerate(pieces):
            docs.append({
                "id": f"rpp-{i}-{j}",
                "text": piece,
                "metadata": {
                    "title": row["title"],
                    "description": row["description"],
                    "link": row["link"],
                    "date_published": row["date_published"]
                }
            })
    return docs

docs = build_docs(df_raw)
len(docs)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

50

## 6) Crear colección Chroma con LangChain

In [7]:

def build_vectorstore_langchain(docs: List[Dict[str, Any]], collection_name="rpp_news_langchain") -> Chroma:
    vectorstore = Chroma(collection_name=collection_name, embedding_function=STEmbeddings())
    vectorstore.add_texts(
        texts=[d["text"] for d in docs],
        metadatas=[d["metadata"] for d in docs],
        ids=[d["id"] for d in docs]
    )
    return vectorstore

def build_retriever(vectorstore: Chroma, k: int = 8):
    return vectorstore.as_retriever(search_kwargs={"k": k})

vectorstore = build_vectorstore_langchain(docs)
retriever = build_retriever(vectorstore, k=8)
print("Documentos indexados:", len(docs))


Documentos indexados: 50


## 7) Consulta de ejemplo

In [8]:

def query_news(retriever, query: str, k: int = 8) -> pd.DataFrame:
    docs_found = retriever.get_relevant_documents(query)
    rows = []
    for d in docs_found:
        m = d.metadata or {}
        rows.append({
            "title": m.get("title",""),
            "description": m.get("description",""),
            "link": m.get("link",""),
            "date_published": m.get("date_published","")
        })
    return pd.DataFrame(rows, columns=["title","description","link","date_published"])

df_results = query_news(retriever, "Últimas noticias de economía", k=8)
df_results


/tmp/ipython-input-4065674914.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs_found = retriever.get_relevant_documents(query)


,title,description,link,date_published
0,¿Seguirá cayendo el dólar o se estabilizará? E...,El dólar acumula una fuerte caída en las últim...,https://rpp.pe/economia/economia/precio-del-do...,"Fri, 17 Oct 2025 18:00:31 -0500"
1,Tumbes: Fiscalía inició una investigación cont...,"Según la Policía Nacional, el mototaxista se n...",https://rpp.pe/peru/tumbes/tumbes-que-pasara-c...,"Fri, 17 Oct 2025 16:45:14 -0500"
2,¿Las deudas por préstamos o tarjeta pueden gen...,"Jhancarlos Palomino, abogado civil y comercial...",https://rpp.pe/economia/tus-finanzas/las-deuda...,"Fri, 17 Oct 2025 16:00:28 -0500"
3,Narváez califica de “injustificada” su salida ...,"Alejandro Narváez, expresidente de Petroperú, ...",https://rpp.pe/economia/economia/petroperu-nar...,"Fri, 17 Oct 2025 18:00:16 -0500"
4,Fiscalía solicitó la detención preliminar de d...,Los suboficiales Luis Magallanes y Omar Saaved...,https://rpp.pe/lima/actualidad/eduardo-ruiz-fi...,"Fri, 17 Oct 2025 17:00:23 -0500"
5,'Agustín' corto escolar del proyecto Creativit...,Producción realizada por los colegios Villa Ca...,https://rpp.pe/columnistas/ursuladelgado/agust...,"Fri, 17 Oct 2025 17:23:58 -0500"
6,La anomia en el tejido social peruano: ¿en el ...,"La anomia, fenómeno caracterizado por la falta...",https://rpp.pe/columnistas/ricardoleninalfredo...,"Fri, 17 Oct 2025 18:36:29 -0500"
7,"María Corina Machado dice que Venezuela ""está ...",La líder política denunció la censura y repres...,https://rpp.pe/mundo/actualidad/maria-corina-m...,"Fri, 17 Oct 2025 16:18:00 -0500"


## 8) Guardar resultados en CSV

In [9]:

out_path = "outputs/query_economia.csv"
df_results.to_csv(out_path, index=False)
print("CSV guardado en:", out_path)


CSV guardado en: outputs/query_economia.csv


## 9) Pipeline completo end-to-end

In [10]:

def pipeline_end_to_end(query="Últimas noticias de política", k=6) -> pd.DataFrame:
    df = load_rss_rpp()
    docs = build_docs(df)
    vs = build_vectorstore_langchain(docs, collection_name="rpp_news_pipeline")
    ret = build_retriever(vs, k=k)
    return query_news(ret, query, k=k)

df_pipeline = pipeline_end_to_end()
df_pipeline


,title,description,link,date_published
0,"María Corina Machado dice que Venezuela ""está ...",La líder política denunció la censura y repres...,https://rpp.pe/mundo/actualidad/maria-corina-m...,"Fri, 17 Oct 2025 16:18:00 -0500"
1,Elecciones 2026: estas agrupaciones políticas ...,De las 39 agrupaciones inscritas para las Elec...,https://rpp.pe/politica/elecciones/elecciones-...,"Fri, 17 Oct 2025 14:51:45 -0500"
2,"""Hay un pleno respaldo"": José Jerí felicita a ...",El presidente José Jerí agradeció a los oficia...,https://rpp.pe/politica/gobierno/jose-jeri-fel...,"Fri, 17 Oct 2025 17:40:26 -0500"
3,¿Seguirá cayendo el dólar o se estabilizará? E...,El dólar acumula una fuerte caída en las últim...,https://rpp.pe/economia/economia/precio-del-do...,"Fri, 17 Oct 2025 18:00:31 -0500"
4,Poder Judicial evalúa pedido fiscal de detenci...,Los policías Luis Magallanes y Omar Saavedra a...,https://rpp.pe/politica/judiciales/caso-eduard...,"Fri, 17 Oct 2025 21:30:15 -0500"
5,Fiscalía solicitó la detención preliminar de d...,Los suboficiales Luis Magallanes y Omar Saaved...,https://rpp.pe/lima/actualidad/eduardo-ruiz-fi...,"Fri, 17 Oct 2025 17:00:23 -0500"



## ✅ Checklist de rúbrica
- RSS parseado (RPP)  
- Tokenización con `tiktoken`  
- Embeddings con `all-MiniLM-L6-v2`  
- ChromaDB con LangChain (store + upsert + retrieval)  
- Orquestación modular  
- Tabla con 4 columnas (title, description, link, date_published)  
- CSV en `outputs/`  
